In [1]:
\cd ../qenv/

In [37]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"37) .engine.logic.inventory.Fill (UNIT) :READY"
`Function for deriving the exec cost from the qty and the price
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
-------------------------------------------------------------
256    ERROR INVERSE:flat to long: UPL: 0, RPL:0 ONE POSITION
"                                                                                                                                                                                                                                                

In [13]:
iv:first flip .bam.x

In [27]:
.engine.model.inventory.Inventory

aId side| ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| -----------------------------------------------------------------------------
0   -1  | 0             0       10     0                                10     0       
0   1   | 0             0       10     0                                10     0       


In [17]:
.engine.logic.contract.UnrealizedPnl    :{[contractType;markPrice;faceValue;multiplier;amt;isignum;avgPrice] 
      7h$($[contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
            contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
          contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
      'nyi] | 0)
    };

In [16]:
.engine.logic.contract.RealizedPnl    :{[contractType;fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier]
    7h$($[contractType=0;.engine.logic.contract.inverse.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=1;.engine.logic.contract.linear.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=2;.engine.logic.contract.quanto.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      'nyi] | 0)
    };

In [47]:
select ivId.ordVal from .bam.x

ordVal|  


In [ ]:
.engine.logic.contract.ExecCost    :{[contractType;price;qty;multiplier]

fId     | 0          1         
price   | 1000       1000      
qty     | 100        100       
reduce  | 0          1         
ismaker | 0          1         
side    | 1          -1        
iId     | 0          0         
oId     | 0          1         
ivId    | 1          0         
aId     | 0          0         
avgPrice| 0          0         
execCost| 10000000   10000000  
ordLoss | 0          0         
ordVal  | -100000000 -100000000
ordQty  | -100       -100      
amt     | 110        110       
totEnt  | 210        210       


In [39]:
iviv:![![.bam.x;();0b;`avgPrice`execCost`ordLoss`ordVal`ordQty`amt`totEnt!(
        ((';`.engine.logic.contract.AvgPrice);`iId.cntTyp;`side;`ivId.execCost;`ivId.totEnt;`iId.smul); // TODO dependent
        ((';`.engine.logic.contract.ExecCost);`iId.cntTyp;`price;`qty;`iId.smul);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;`iId.mkprice;`qty;((+/);`qty;`price)));
        (-;`ivId.ordVal;((*/);`qty;`price));
        (-;`ivId.ordQty;`qty);
        (+;`ivId.amt;`qty);
        ((+/);`ivId.totEnt;`qty)
    )];
    ();0b;`upnl`rpnl!(
    ((';`.engine.logic.contract.UnrealizedPnl);`iId.cntTyp;`iId.mkprice;`iId.faceValue;`iId.smul;`ivId.amt;`side;`ivId.avgPrice);
    ((';`.engine.logic.contract.RealizedPnl);`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul))]

In [59]:
.q.cl:{x!y[x]}

In [67]:
.engine.model.inventory.Inventory upsert flip 

aId side| ordQty ordVal     ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| ---------------------------------------------------------------------------------
0   -1  | -100   -100000000 0       110    0         0    0        10000000 210    0       
0   1   | -100   -100000000 0       110    0         0    0        10000000 210    0       


In [68]:
.bam.x

fId    | 0    1   
price  | 1000 1000
qty    | 100  100 
reduce | 0    1   
ismaker| 0    1   
side   | 1    -1  
iId    | 0    0   
oId    | 0    1   
ivId   | 1    0   
aId    | 0    0   


In [ ]:
.engine.logic.inventory.Fill:{ // TODO simple select

    iv:?[?[x;();0b;(
                (`.engine.logic.contract.UnrealizedPnl;`cntTyp;`mkprice;`faceValue;`smul;`amt;`side;`avgPrice);
                (`.engine.logic.contract.RealizedPnl;`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul)
        )];();0b;(
        (`.engine.logic.contract.AvgPrice);
        (`.engine.logic.contract.ExecCost);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;));
        (-;`ivId.ordVal;(`.engine.logic.contract.Value;));
        (-;`ivId.ordQty;qty);
        (+;`amt;`dlt);
        (sum;`ivId.totalEntry;(max;`dlt;0))
        )];

    .engine.model.inventory.Update iv;
    .engine.EmitA[`inventory;];

    		/if[abs[iv[`amt]]=0;iv[`avgPrice`execCost`totalEntry]:0];

    };



In [40]:
e:flip(iv`kind;iv`time;flip iv`aId`side`time`amt`rpnl`avgPrice`upnl;7h$iv`aId)

In [44]:
x:.bam.x

In [46]:
x[`kind]:`fill

In [47]:
e:flip(x`kind;x`time;flip x`time`price`qty`side;7h$x`aId)

In [50]:
.engine.egress.Events

time                    kind      datum                                                                 aId
-----------------------------------------------------------------------------------------------------------
2020.12.13T14:00:29.774 inventory `.engine.model.account.Account$0 1  2020.12.13T14:00:29.774 110 0 0 0 0  
2020.12.13T14:00:29.774 inventory `.engine.model.account.Account$0 -1 2020.12.13T14:00:29.774 110 0 0 0 0  


In [58]:
e

`fill 2020.12.13T14:00:29.774 (2020.12.13T14:00:29.774;1000;100;1)  0
`fill 2020.12.13T14:00:29.774 (2020.12.13T14:00:29.774;1000;100;-1) 0


In [66]:
m:x

In [71]:
x:m

In [67]:
x:first flip x

In [73]:
x:flip x

In [80]:
x`time;x`time`price`qty`side

2020.12.13T14:00:29.774 2020.12.13T14:00:29.774
1000                    1000                   
100                     100                    
1                       -1                     


In [86]:
.engine.EmitA'[x`kind;scl[`time`price`qty`side;flip x];7h$x`aId];

In [83]:
delete from `.engine.egress.Events

`.engine.egress.Events


In [87]:
.engine.egress.Events

time kind datum aId
-------------------


In [4]:
iv[`kind]:`inventory;
show ;
.engine.EmitA[iv`kind;s`time;flip s`aId`side`time`amt`rpnl`avgPrice`upnl;iv`aId];

[0;31ms[0m: [0;31ms[0m

In [ ]:
.event.Event:{

    }

In [ ]:
.model.Account: {

    }

In [ ]:
.state.AccountEventHistory: (
    [aId : `long$(); time : `datetime$()]
    balance             : `long$();
    available           : `long$();
    frozen              : `long$();
    maintMargin         : `long$());

In [ ]:
.state.InventoryEventHistory: ( // TODO change side to long 1,2,3
    [accountId:  `long$();side: `long$();time : `datetime$()]
    amt                 :  `long$();
    realizedPnl         :  `long$();
    avgPrice            :  `long$(); // TODO check all exchanges have
    unrealizedPnl       :  `long$());

In [144]:
e:.event.Inventory[iv]

[0;31mlength[0m: [0;31mlength[0m

In [158]:
.event.RmFk[flip iv]

time                    datum                                                                    aId kind     
--------------------------------------------------------------------------------------------------------------
2020.12.13T13:51:26.944 `.engine.model.account.Account$0 1  2020.12.13T13:51:26.944 210 0 2100 0 0   inventory
2020.12.13T13:51:26.944 `.engine.model.account.Account$0 -1 2020.12.13T13:51:26.944 210 0 2100 0 0   inventory


fId time                    price qty reduce ismaker side iId oId ivId aId avgPrice execCost ordLoss ordVal     ordQty amt totEnt upnl rpnl kind     
-----------------------------------------------------------------------------------------------------------------------------------------------------
0   2020.12.13T13:51:26.944 1000  100 0      0       1    0   0   1    0   2100     10000000 0       -200000000 -200   210 410    0    0    inventory
1   2020.12.13T13:51:26.944 1000  100 1      1       -1   0   1   0    0   2100     10000000 0       -200000000 -200   210 410    0    0    inventory


In [130]:
.state.OrderEventHistory: (
    [orderId:`long$();time:`datetime$()]
    accountId       :   `long$();
    side            :   `long$(); // TODO change to long
    otype           :   `long$(); // TODO change to long
    price           :   `long$();
    leaves          :   `long$();
    limitprice      :   `long$(); / multiply by 100
    stopprice       :   `long$(); / multiply by 100
    status          :   `long$();
    reduce          :   `boolean$();
    trigger         :   `long$(); // TODO change to long
    execInst        :   `long$()); // TODO change to long
.state.ordCols:cols .state.OrderEventHistory;
.state.CurrentOrders: `orderId xkey .state.OrderEventHistory;

In [131]:
fkeys .state.CurrentOrders

In [169]:
t

In [167]:
.event.RmFk : {fk:key fkeys x;x[fk]:7h$x[fk];x};
.event.Event: {e:flip `time`datum`aId!y;e[`kind]:x;e};
.event.Failure :{()}
.event.Account : {.event.Event[`account;((flip .event.RmFk[flip x]`time;flipx`aId`time`bal`avail`froz);7h$x`aId)]}
.event.Inventory :{.event.Event[`inventory;(x`time;(flip .event.RmFk[flip x]`aId`side`time`amt`rpnl`avgPrice`upnl);7h$x`aId)]}
.event.Order     :{.event.Event[`order;(x`time;(flip .event.RmFk[flip x]`oId`time`aId`amt`rpnl`avgPrice`upnl);7h$x`aId)]}
.event.Deposit :{}
.event.Withdraw :{}
.event.Funding :{}
.event.Mark:{}
.event.Settlement:{}
.event.PriceLimit :{}
.event.Level:{}
.event.Trade:{}
.event.Order:{}
.event.Instrument:{}
.event.Fill:{}
.event.Liquidation:{}

In [ ]:
.util.testutils.makeDefaultsRecords  :{[tab;cl;vl] // TODO inter with actual cols
    r:.util.NullRowDict[tab];
    cvl:count[vl]; 
    :$[cvl>1;[rx:(cvl#enlist[r]);rx[cl]:flip[vl];:rx];[r[cl]:first[vl];:r]]};

In [173]:
.model.Fill:{
    c:x!y;
    c[`time]:x`time;
    c[`price`qty`reduce`ismaker`side`iId`oId`ivId`aId]:x`datum;
    c[`aId]
    };

In [174]:
.fill.r:.util.NullRowDict[`.engine.model.fill.Fill]

In [187]:
.model.Fill  :{[cl;vl] // TODO inter with actual cols
    };

In [178]:
x:.util.testutils.makeDefaultsRecords[`fId`price`side`qty`reduce`ismaker`oId`aId`iId`time;flip(0 1;1000 1000;1 -1;100 100;01b;01b;0 1;0 0;0 0;2#z)]

In [184]:
.engine.model.inventory.Inventory

aId side| ordQty ordVal     ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| ---------------------------------------------------------------------------------
0   -1  | -100   -100000000 0       110    0         0    0        10000000 210    0       
0   1   | -100   -100000000 0       110    0         0    0        10000000 210    0       


In [232]:
c:((!) . flip(
  (`account;.model.Account[`aId`avail`bal;enlist(0;0;0)]); 
  (`instrument;.model.Instrument[`iId`cntTyp`faceValue`mkprice`smul;enlist(0;0;1;1000;1)]); 
  (`inventory;.model.Inventory[`aId`side`mm`upnl`ordQty`ordLoss`ordVal`amt`totEnt;flip(0 0;-1 1;0 0;0 0;0 0;0 0;0 0;10 10;10 10)]); 
  (`feetier;.model.Feetier[`ftId`vol`bal`ref;flip(0 1;0 0;0 0;0 0)]); // Update Account
  (`risktier;.model.Risktier[`rtId`amt`lev;flip(0 1;50000 250000;125 100)]);
  (`order;.model.Order[])))

[0;31m.model.Order[0m: [0;31m.model.Order[0m

In [ ]:
.engine.model.instrument.Instrument,:s[`instrument];
.engine.model.account.Account,:s[`account];
.engine.model.inventory.Inventory,:s[`inventory];
.engine.model.feetier.Feetier,:s[`feetier];
.engine.model.risktier.Risktier,:s[`risktier];

In [187]:
.model.Order[`oqty`price`dlt`reduce`dqty;enlist(1;1000;1;1b;1)];

[0;31m.model.Order[0m: [0;31m.model.Order[0m

In [218]:
.model.Model:{[tb;cl;vl]
    cvl:count[vl]; 
    $[cvl>1;[rx:(cvl#enlist[tb]);rx[cl]:flip[vl];:flip rx];[r[cl]:first[vl];:tb]]
    };

In [231]:
.model.Fill:{[cl;vl]
    //if[null cl;cl:key .fill.r]; // TODO check
    x:.model.Model[.fill.r;cl;vl];
    x[`aId]:`.engine.model.account.Account$x[`aId];
    x[`ivId]:`.engine.model.inventory.Inventory$x[`aId];
    x
    };

In [232]:
x:.model.Fill[`fId`price`side`qty`reduce`ismaker`oId`aId`iId`ivId`time;flip(0 1;1000 1000;1 -1;100 100;01b;01b;0 1;0 0;0 0;0 0;2#z)]

[0;31mcast[0m: [0;31mcast[0m

In [232]:
type each first flip x

fId    | -7
time   | -15
price  | -7
qty    | -7
reduce | -1
ismaker| -1
side   | -7
iId    | -7
oId    | -7
ivId   | -7
aId    | -20
